# Import & Explore

In [ ]:
!pip install geemap folium

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import folium
import geemap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path1 = '/content/drive/MyDrive/SPICE/balanced_data.csv'
data = pd.read_csv(path1)

In [ ]:
data.head()

,Location,Date,Sentinel-2,Longitude,Latitude,B1,B2,B3,B4,B5,...,GNDVI,MCARI,MSI,NDMI,NBR,NDSI,SAVI,OSI,PNDVI,label
0,Caribbean Sea,20161103,A,-86.422592,15.947693,0.022240,0.024286,0.022068,0.016566,0.012177,...,-0.261005,-0.002935,1.042256,-0.020691,0.007203,0.241619,-0.010295,1.590786,-0.659011,1
1,Gulf of Gera,20210626,B,26.528152,39.040215,0.016377,0.024589,0.036882,0.013402,0.011666,...,-0.226979,0.002654,1.103045,-0.048998,0.209766,0.179982,0.027489,2.044950,-0.526318,1
2,Gulf of Gera,20210626,B,26.522369,39.041180,0.015475,0.020442,0.020078,0.003485,0.001777,...,-0.227117,0.000157,0.505766,0.328228,0.332796,0.516818,0.026622,1.152688,-0.553553,1
3,Gulf of Gera,20210626,B,26.521099,39.041176,0.015464,0.023483,0.018375,0.002875,0.001410,...,-0.247050,0.000198,0.484931,0.346864,0.338666,0.547037,0.023987,0.904912,-0.602547,1
4,Gulf of Gera,20210626,B,26.525602,39.041553,0.013649,0.017143,0.019180,0.002774,0.001669,...,-0.161507,0.001002,0.543585,0.295685,0.316972,0.436354,0.032148,1.280645,-0.476946,1


#Clean

In [ ]:
features = data[['NDSI', 'MNDWI', 'NDWI', 'OSI', 'FDI', 'B1']]
target_y = data['label']
x_train, x_test, y_train, y_test = train_test_split(features, target_y, test_size = 0.3, random_state = 13)

In [ ]:
model_vml = DecisionTreeClassifier(random_state = 13)
model_vml.fit(x_train, y_train) ## ^

DecisionTreeClassifier(random_state=13)

In [ ]:
y_pred = model_vml.predict(x_test) ## from modelv6
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Model: {accuracy:.2f}")

Accuracy of Model: 0.88


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path2 = '/content/drive/MyDrive/SPICE/senty.csv'
data2 = pd.read_csv(path2)

In [ ]:
features = data2[['NDSI', 'MNDWI', 'NDWI', 'OSI', 'FDI', 'B01_20m']]

In [ ]:
features = features.rename(columns={'B01_20m': 'B1'})
features.head()

,NDSI,MNDWI,NDWI,OSI,FDI,B1
0,0.045398,0.060862,0.046706,1.491884,-11.547282,273
1,0.039158,0.045763,0.035235,1.718460,-116441.602285,252
2,0.033501,0.030480,0.024491,1.735781,-116424.940131,260
3,0.029340,0.019370,0.017728,1.740636,-116414.832029,272
4,0.018853,0.011306,0.008162,1.733379,21.560821,280


In [ ]:
predictions = model_vml.predict(features)

In [ ]:
data3 = data2[['latitude', 'longitude']]
data3['label'] = predictions
data3.head()

<ipython-input-26-227c09d91a82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['label'] = predictions


,latitude,longitude,label
0,21.966985,-157.809977,1
1,21.966983,-157.809783,1
2,21.966982,-157.809590,1
3,21.966981,-157.809396,1
4,21.966979,-157.809202,2


In [ ]:
from google.colab import files

In [ ]:
# Define labels to filter out
labels_to_filter = [1, 3, 4, 5, 6, 7]

# Filter rows where Labels column is not in the list of labels_to_filter
df = data3[~data3['label'].isin(labels_to_filter)]
df.head()

,latitude,longitude,label
4,21.966979,-157.809202,2
121,21.965902,-157.810180,2
619,21.964535,-157.796052,2
653,21.964430,-157.806512,2
943,21.963890,-157.806710,2


In [ ]:
'''df.to_csv('Plastic.csv', index=False)
files.download('Plastic.csv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Mapping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/SPICE/Plastic.csv'
data4 = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from folium.plugins import MarkerCluster
from IPython.display import display

# Create a map centered around the average latitude and longitude
map_center = [data4['latitude'].mean(), data4['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=7)

'''# Initialize the map centered on the midpoint
m = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom_start=10)
'''

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Sample the data to reduce the number of markers
data_snip = data4.sample(frac=0.1)  # Adjust the fraction as needed

# Add markers to the cluster
for idx, row in data_snip.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup='plastic',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

'''# Save the map to an HTML file in Google Drive
m.save('/content/drive/MyDrive/SPICE/Mappy.html')'''

# Display map
m